In [ ]:
library(TCGAbiolinks)
library(DESeq2)
library(ggplot2)

In [ ]:
#download
query.KIRP <- GDCquery(project = "TCGA-KIRP", 
                       data.category = "Transcriptome Profiling", 
                       data.type = "Gene Expression Quantification", 
                       workflow.type = "HTSeq - Counts", 
                       sample.type = "Primary Tumor")

GDCdownload(query.KIRP)
tumor.kirp <- GDCprepare(query = query.KIRP, 
                         save = TRUE, 
                         save.filename = "kirpRNAseqTumor.rda", 
                         summarizedExperiment = TRUE)

In [ ]:
#download
query.KIRP_normal <- GDCquery(project = "TCGA-KIRP", 
                       data.category = "Transcriptome Profiling", 
                       data.type = "Gene Expression Quantification", 
                       workflow.type = "HTSeq - Counts", 
                       sample.type = "Solid Tissue Normal")

GDCdownload(query.KIRP_normal)
tumor.kirp_normal <- GDCprepare(query = query.KIRP_normal, 
                                save = TRUE, 
                                save.filename = "kirpRNAseqNormal.rda", 
                                summarizedExperiment = TRUE)

In [ ]:
(tumor.kirp_normal)
print("")
(tumor.kirp)

In [ ]:
(colData(tumor.kirp_normal)$Normal = factor(1, levels = c(1, 0)))
(colData(tumor.kirp)$Normal = factor(0, levels = c(1, 0)))

In [ ]:
commoncoldata <- intersect(names(colData(tumor.kirp)), names(colData(tumor.kirp_normal)))
colData(tumor.kirp) <- colData(tumor.kirp)[commoncoldata]
(colData(tumor.kirp_normal) <- colData(tumor.kirp_normal)[commoncoldata])

In [ ]:
kirp_combined <- SummarizedExperiment::cbind(tumor.kirp_normal, tumor.kirp)

In [ ]:
#Design
kirp_combined.dds <- DESeqDataSet(kirp_combined, design = ~ Normal)

In [ ]:
# Remove sample with less than 10 counts across samples for a given gene
kirp_combined.dds <- kirp_combined.dds[rowSums(counts(kirp_combined.dds))>10]

In [ ]:
dds_kirp <- DESeq(kirp_combined.dds)

In [ ]:
res <- results(dds_kirp)
mcols(res)
summary(res)

In [ ]:
filter_upgenes = which(res$padj < 0.05 & res$log2FoldChange > 0)
total_upgenes = length(filter_upgenes)
filtered_up_res = res[filter_upgenes,]

filter_downgenes = which(res$padj < 0.05 & res$log2FoldChange < 0)
total_downgenes = length(filter_downgenes)
filtered_down_res = res[filter_downgenes,]

In [ ]:
percent_genes_proposed = 0.5

In [ ]:
(lfc_5000_up_thresh = sort(filtered_up_res$log2FoldChange, decreasing = TRUE)[ percent_genes_proposed*total_upgenes ])
(lfc_5000_down_thresh = sort(filtered_down_res$log2FoldChange, decreasing = FALSE)[percent_genes_proposed*total_downgenes])

In [ ]:
#upgenes
indices <- which(res$padj < 0.05 & res$log2FoldChange > 1)
up_res <- res[indices, ] 
upgenes <- rownames(head(res[order(res$log2FoldChange, decreasing = TRUE), ], n=length(indices)))
print("num upgenes")
print(length(upgenes)) 

In [ ]:
#upgenes
# indices <- which(res$padj < 0.05 & res$log2FoldChange >  lfc_5000_up_thresh)
# up_res <- res[indices, ]
# upgenes <- rownames(head(res[order(res$log2FoldChange, decreasing = TRUE), ], n=length(indices)))
# print("num upgenes")
# print(length(upgenes)) 

In [ ]:
#downgenes
indices <- which(res$padj < 0.05 & res$log2FoldChange < -1)
down_res <- res[indices, ]
downgenes <- rownames(head(down_res[order(-down_res$log2FoldChange, decreasing = TRUE), ], n=length(indices)))
print("num downgenes")
print(length(downgenes)) 

In [ ]:
#downgenes
# indices <- which(res$padj < 0.05 & res$log2FoldChange < lfc_5000_down_thresh)
# down_res <- res[indices, ]
# downgenes <- rownames(head(down_res[order(-down_res$log2FoldChange, decreasing = TRUE), ], n=length(indices)))
# print("num downgenes")
# print(length(downgenes)) 

In [ ]:
# Extracting genenames
library("AnnotationDbi")
library("org.Hs.eg.db")

In [ ]:
#upgenes
genenames <- mapIds(org.Hs.eg.db,
                     #keys=upgenes,
                     keys = as.character(upgenes),
                     column="SYMBOL",
                     keytype="ENSEMBL",
                     multiVals="first")

map.df = as.data.frame(as.matrix(genenames))
name=map.df$V1

In [ ]:
sum(!is.na(name))

In [ ]:
write.csv(name,'up_kirp_genes.csv', sep = '\t')

In [ ]:
#downgenes
genenames1 <- mapIds(org.Hs.eg.db,
                    #keys=upgenes,
                    keys = as.character(downgenes),
                    column="SYMBOL",
                    keytype="ENSEMBL",
                    multiVals="first")

map.df1 = as.data.frame(as.matrix(genenames1))
name1=map.df1$V1

In [ ]:
sum(!is.na(name1))

In [ ]:
write.csv(name1,'down_kirp_genes.csv', sep = '\t')